In [1]:
!pip install transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import transformers
import torch
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
dataset = pd.read_csv('Classified_Dataset.csv')

# Extract the columns for xi, xj, and model (LLM label)
xi = dataset['xi'].tolist()  # truncated statement
xj = dataset['xj'].tolist()  # generated statement
labels = dataset['model'].tolist()  # LLM label

In [5]:
model = transformers.AutoModel.from_pretrained("jxm/cde-small-v1", trust_remote_code=True)
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

misc.py:   0%|          | 0.00/720 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jxm/cde-small-v1:
- misc.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.py:   0%|          | 0.00/40.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
xi_prefix = "truncated_statement: "
xj_prefix = "generated_statement: "

# Device setup (e.g., using GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
def get_embeddings(texts, prefix, model, tokenizer, batch_size=4): # Reduced batch size
    input_texts = [prefix + str(text) for text in texts]
    input_docs = tokenizer(
        input_texts,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    model = model.to(device)

    # First stage embedding
    dataset_embeddings = []
    for i in tqdm(range(0, len(input_docs["input_ids"]), batch_size)):
        input_docs_batch = {k: v[i:i+batch_size].to(device) for k, v in input_docs.items()}
        with torch.no_grad():
            outputs = model.first_stage_model(**input_docs_batch)
            dataset_embeddings.append(outputs)
            del outputs # Delete outputs after appending to free up memory
            torch.cuda.empty_cache() # Empty cache to further reduce memory usage

    dataset_embeddings = torch.cat(dataset_embeddings).to(device)

    # Second stage embedding
    final_embeddings = []
    for i in tqdm(range(0, len(input_docs["input_ids"]), batch_size)): # Process second stage in batches
        dataset_embeddings_batch = dataset_embeddings[i:i+batch_size].to(device)
        input_docs_batch = {k: v[i:i+batch_size].to(device) for k, v in input_docs.items()}
        with torch.no_grad():
            outputs = model.second_stage_model(
                input_ids=input_docs_batch["input_ids"],
                attention_mask=input_docs_batch["attention_mask"],
                dataset_embeddings=dataset_embeddings_batch
            )
            final_embeddings.append(outputs)
            del outputs # Delete outputs after appending to free up memory
            torch.cuda.empty_cache() # Empty cache to further reduce memory usage

    final_embeddings = torch.cat(final_embeddings).to(device)

    # Normalize embeddings
    final_embeddings = final_embeddings / final_embeddings.norm(p=2, dim=1, keepdim=True)

    return final_embeddings

In [8]:
xi_train, xi_test, xj_train, xj_test, labels_train, labels_test = train_test_split(
    xi, xj, labels, test_size=0.2, shuffle=False)

# Step 1: Get embeddings for xi and xj (training and testing)
xi_train_embeddings = get_embeddings(xi_train, xi_prefix, model, tokenizer)
xj_train_embeddings = get_embeddings(xj_train, xj_prefix, model, tokenizer)

xi_test_embeddings = get_embeddings(xi_test, xi_prefix, model, tokenizer)
xj_test_embeddings = get_embeddings(xj_test, xj_prefix, model, tokenizer)


  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

In [9]:
combined_train_embeddings = torch.cat((xi_train_embeddings, xj_train_embeddings), dim=1)
combined_test_embeddings = torch.cat((xi_test_embeddings, xj_test_embeddings), dim=1)


In [10]:
label_encoder = LabelEncoder()
encoded_train_labels = torch.tensor(label_encoder.fit_transform(labels_train)).to(device)
encoded_test_labels = torch.tensor(label_encoder.transform(labels_test)).to(device)


In [11]:
import torch.nn as nn

In [12]:
class ImprovedLLMClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ImprovedLLMClassifier, self).__init__()

        # First hidden layer with batch normalization
        self.fc1 = nn.Linear(input_dim, 1024)  # Increased neurons
        self.bn1 = nn.BatchNorm1d(1024)

        # Second hidden layer with batch normalization
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)

        # Third hidden layer with batch normalization
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)

        # Output layer
        self.fc4 = nn.Linear(256, num_classes)

        # Activation function
        self.relu = nn.ReLU()

        # Dropout to prevent overfitting
        self.dropout = nn.Dropout(0.2)  # Lower dropout rate

    def forward(self, x):
        # Pass through first hidden layer
        x = self.relu(self.bn1(self.fc1(x)))

        # Apply dropout
        x = self.dropout(x)

        # Pass through second hidden layer
        x = self.relu(self.bn2(self.fc2(x)))

        # Apply dropout
        x = self.dropout(x)

        # Pass through third hidden layer
        x = self.relu(self.bn3(self.fc3(x)))

        # Apply dropout
        x = self.dropout(x)

        # Output layer
        x = self.fc4(x)

        return x

In [13]:
input_dim = combined_train_embeddings.size(1)
num_classes = len(label_encoder.classes_)

In [14]:
classifier = ImprovedLLMClassifier(input_dim=input_dim, num_classes=num_classes).to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)


In [15]:
num_epochs = 75
batch_size = 64

# Training phase
for epoch in range(num_epochs):
    classifier.train()
    for i in range(0, len(combined_train_embeddings), batch_size):
        # Batch data
        batch_embeddings = combined_train_embeddings[i:i+batch_size]
        batch_labels = encoded_train_labels[i:i+batch_size]

        # Forward pass
        outputs = classifier(batch_embeddings)
        loss = criterion(outputs, batch_labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluation phase (on the test set)
classifier.eval()

Epoch [1/75], Loss: 1.1911
Epoch [2/75], Loss: 1.0036
Epoch [3/75], Loss: 0.9155
Epoch [4/75], Loss: 0.8323
Epoch [5/75], Loss: 0.6980
Epoch [6/75], Loss: 0.5516
Epoch [7/75], Loss: 0.5938
Epoch [8/75], Loss: 0.4344
Epoch [9/75], Loss: 0.3077
Epoch [10/75], Loss: 0.2675
Epoch [11/75], Loss: 0.1453
Epoch [12/75], Loss: 0.2647
Epoch [13/75], Loss: 0.1981
Epoch [14/75], Loss: 0.0677
Epoch [15/75], Loss: 0.2504
Epoch [16/75], Loss: 0.1026
Epoch [17/75], Loss: 0.1035
Epoch [18/75], Loss: 0.1487
Epoch [19/75], Loss: 0.1718
Epoch [20/75], Loss: 0.0528
Epoch [21/75], Loss: 0.0252
Epoch [22/75], Loss: 0.1252
Epoch [23/75], Loss: 0.1482
Epoch [24/75], Loss: 0.0502
Epoch [25/75], Loss: 0.0330
Epoch [26/75], Loss: 0.1305
Epoch [27/75], Loss: 0.0271
Epoch [28/75], Loss: 0.0625
Epoch [29/75], Loss: 0.0116
Epoch [30/75], Loss: 0.1234
Epoch [31/75], Loss: 0.1053
Epoch [32/75], Loss: 0.0131
Epoch [33/75], Loss: 0.0419
Epoch [34/75], Loss: 0.0961
Epoch [35/75], Loss: 0.1106
Epoch [36/75], Loss: 0.0824
E

ImprovedLLMClassifier(
  (fc1): Linear(in_features=1536, out_features=1024, bias=True)
  (bn1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=256, out_features=6, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [16]:
with torch.no_grad():
    test_outputs = classifier(combined_test_embeddings)
    test_loss = criterion(test_outputs, encoded_test_labels)

    # Compute accuracy
    _, predicted = torch.max(test_outputs, 1)
    correct = (predicted == encoded_test_labels).sum().item()
    accuracy = correct / len(encoded_test_labels)
    print(f"Test Loss: {test_loss.item():.4f}, Test Accuracy: {accuracy:.4f}")

Test Loss: 3.1643, Test Accuracy: 0.5645


In [19]:
from sklearn.metrics import classification_report

# Assuming `encoded_test_labels` is your true labels
# and `predicted` is your predicted labels extracted from `test_outputs`
# Replace `y_test` and `y_pred` with your actual variable names

# Get predicted labels from test_outputs
_, predicted = torch.max(test_outputs, 1)

# Generate and display the classification report
print("Classification Report:")
print(classification_report(encoded_test_labels.cpu(), predicted.cpu())) # Move tensors to CPU for sklearn

Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.55      0.54      1000
           1       0.54      0.53      0.54      1000
           2       0.55      0.52      0.53      1000
           3       0.44      0.49      0.47      1000
           4       0.71      0.71      0.71      1000
           5       0.63      0.58      0.60      1000

    accuracy                           0.56      6000
   macro avg       0.57      0.56      0.57      6000
weighted avg       0.57      0.56      0.57      6000

